In [1]:
immutable SkipWrapIterator{T}
    source::T
    skip::Integer
end

Base.eltype{T}(::SkipWrapIterator{T}) = Base.eltype(T)


  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31


In [2]:
Base.eltype{T}(::SkipWrapIterator{T}) = Base.eltype(T)
function Base.iteratorsize{T}(::SkipWrapIterator{T}) 
    source_itersize = Base.iteratorsize(T)
    if source_itersize == Base.HasShape() || source_itersize == Base.HasLength()
        Base.HasLength() #Never give it a shape
    else
        Base.SizeUnknown()
    end
end

Base.length(swi::SkipWrapIterator) = Base.length(swi.source)

In [3]:
function Base.start(swi::SkipWrapIterator)
    source_initial_state = Base.start(swi.source)
    source_state = deepcopy(source_initial_state)
    for ii in 1:swi.skip
        item, source_state = Base.next(swi.source, source_state)
    end
            
    wrapback_progress = -1
    (source_state, source_initial_state, wrapback_progress)
end

function Base.next(swi::SkipWrapIterator, state)
    (source_state, source_initial_state, wrapback_progress) = state
    if Base.done(swi.source, source_state) 
        #Do the wrap back
        @assert (wrapback_progress == -1) 
        # This is our first time back
        #println("Began wrap")
        wrapback_progress = 0
        source_state = source_initial_state
    end
    if wrapback_progress >= 0
        wrapback_progress += 1
    end
    #@show source_state
    
    item, source_state = Base.next(swi.source, source_state)
    #@show (item,source_state)
    item, (source_state, source_initial_state, wrapback_progress)
end

function Base.done(swi::SkipWrapIterator, state_d)
    #@show state_d
       
    (source_state, source_initial_state, wrapback_progress) = state_d
    if swi.skip == 0
        Base.done(swi.source, source_state)
    else
        wrapback_progress >= swi.skip
    end
end

In [4]:
using Base.Test
@testset "Wrap Back Around Iterator" begin
    @test collect(SkipWrapIterator([1,2,3,4,5], 0))== [1,2,3,4,5]
    @test collect(SkipWrapIterator([1,2,3,4,5], 1)) == [2,3,4,5,1] 
    @test collect(SkipWrapIterator([1,2,3,4,5], 2)) == [3,4,5,1,2] 
    @test collect(SkipWrapIterator([1,2,3,4,5], 3)) == [4,5,1,2,3] 

    @test (collect(SkipWrapIterator(eachline(IOBuffer("a\nb\nc\nd\ne\nf\n")),0)) ==
        ["a\n", "b\n", "c\n", "d\n", "e\n", "f\n"])
    
    @test (collect(SkipWrapIterator(eachline(IOBuffer("a\nb\nc\nd\ne\nf\n")),3)) ==
        ["d\n", "e\n", "f\n", "a\n", "b\n", "c\n"])
end

Wrap Back Around Iterator: Error During Test
  Test threw an exception of type AssertionError
  Expression: collect(SkipWrapIterator(eachline(IOBuffer("a\nb\nc\nd\ne\nf\n")),3)) == ["d\n","e\n","f\n","a\n","b\n","c\n"]
  AssertionError: wrapback_progress == -1
   in next(::SkipWrapIterator{EachLine}, ::Tuple{Void,Void,Int64}) at ./In[3]:16
   in _collect(::UnitRange{Int64}, ::SkipWrapIterator{EachLine}, ::Base.HasEltype, ::Base.SizeUnknown) at ./array.jl:241
   in collect(::SkipWrapIterator{EachLine}) at ./array.jl:232
   in macro expansion; at ./In[4]:11 [inlined]
   in macro expansion; at ./test.jl:672 [inlined]
   in anonymous at ./<missing>:?
   in include_string(::String, ::String) at ./loading.jl:380
   in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:183
   in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
   in (::IJulia.##24#30)() at ./task.jl:309
Test Summary:             | Pass  

LoadError: LoadError: Some tests did not pass: 5 passed, 0 failed, 1 errored, 0 broken.
while loading In[4], in expression starting on line 2

In [5]:
using WordStreams
@show typeof(words_of(IOBuffer("a b c d e f")))

Base.iteratorsize(typeof(words_of(IOBuffer("a b c d e f"))))

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in _collect(::Array{SubString{String},1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Base.EltypeUnknown, ::Base.HasShape) at ./array.jl:258
 in #error_content#12(::Symbol, ::String, ::Function, ::LoadError, ::Array{Ptr{Void},1}) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:75
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:238
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/v0.5/IJulia/src/IJulia.jl:143
 in (::IJulia.##24#30)() at ./task.jl:309
while loading /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl, in expression starting on line 31
 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::SubString{String}) at ./deprecated.jl:50
 in collect_to!(::Array{String,1}, ::Base.Generator{Array{SubString{String},1},Base.#utf8}, ::Int64, ::Int64) at ./array.jl:277
 in _collect(:

typeof(words_of(IOBuffer("a b c d e f"))) = WordStreams.WordStream{String,Float32}


  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
  likely near /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:480
in Type at /home/ubuntu/.julia/v0.5/PooledElements/src/pool.jl


Base.SizeUnknown()

In [6]:
Base.iteratorsize(WordStream{String,Float32})

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


Base.SizeUnknown()

In [16]:
collect(SkipWrapIterator(words_of(open("eg.txt","r")),0))

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in utf8(::String) at ./deprecated.jl:50
 in pstring at /home/ubuntu/.julia/v0.5/PooledElements/src/pooledstring.jl:0 [inlined]
 in unrated_next_word!(::WordStreams.WordStream{String,Float32}, ::IOStream) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordStreams.jl:54
 in start(::WordStreams.WordStream{String,Float32}) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordStreams.jl:73
 in start(::SkipWrapIterator{WordStreams.WordStream{String,Float32}}) at ./In[3]:2
 in _collect(::UnitRange{Int64}, ::SkipWrapIterator{WordStreams.WordStream{String,Float32}}, ::Base.HasEltype, ::Base.SizeUnknown) at ./array.jl:241
 in collect(::SkipWrapIterator{WordStreams.WordStream{String,Float32}}) at ./array.jl:232
 in include_string(::String, ::String) at ./loading.jl:380
 in execute_request_0x535c5df2(::ZMQ.Socket, ::IJulia.Msg) at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl:183
 in eventloop(::ZMQ.Socket) at /home/ubuntu/.julia/

6-element Array{String,1}:
 "a"
 "b"
 "c"
 "d"
 "e"
 "f"

In [17]:
collect(SkipWrapIterator(words_of(open("eg.txt","r")),3))

6-element Array{String,1}:
 "d"
 "e"
 "f"
 "a"
 "b"
 "c"

In [ ]:
Base.it

In [ ]:
methods(Base.iteratorsize)

# 32 methods for generic function "iteratorsize":
iteratorsize(::Type{EachLine}) at io.jl:383
iteratorsize(::Type{Base.RegexMatchIterator}) at regex.jl:325
iteratorsize(::Type{Task}) at task.jl:242
iteratorsize(::Type{Base.AsyncGenerator}) at asyncmap.jl:169
iteratorsize(::Type{Base.LibGit2.GitBranchIter}) at libgit2/reference.jl:215
iteratorsize(::Type{Base.LibGit2.GitRevWalker}) at libgit2/walker.jl:28
iteratorsize(::Type{WordStreams.SlidingWindow}) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordStreams.jl:111
iteratorsize{T<:AbstractArray{T,N}}(::Type{T}) at generator.jl:54
iteratorsize{I,F}(::Type{Base.Generator{I,F}}) at generator.jl:55
iteratorsize{T<:Number}(::Type{T}) at number.jl:16
iteratorsize{I}(::Type{Enumerate{I}}) at iterator.jl:33
iteratorsize{I}(::Type{Base.Zip1{I}}) at iterator.jl:60
iteratorsize{I1,I2}(::Type{Base.Zip2{I1,I2}}) at iterator.jl:79
iteratorsize{I1,I2}(::Type{Zip{I1,I2}}) at iterator.jl:103
iteratorsize{T<:Filter{F,I}}(::Type{T}) at iterator.jl:144
iteratorsize{I,S}(::Type{Base.Rest{I,S}}) at iterator.jl:162
iteratorsize{S}(::Type{Base.Count{S}}) at iterator.jl:199
iteratorsize{I}(::Type{Base.Take{I}}) at iterator.jl:213
iteratorsize{I}(::Type{Base.Drop{I}}) at iterator.jl:242
iteratorsize{I}(::Type{Base.Cycle{I}}) at iterator.jl:269
iteratorsize{O}(::Type{Base.Repeated{O}}) at iterator.jl:302
iteratorsize{I}(::Type{Base.Prod1{I}}) at iterator.jl:343
iteratorsize{I1,I2}(::Type{Base.Prod2{I1,I2}}) at iterator.jl:372
iteratorsize{I1,I2}(::Type{Base.Prod{I1,I2}}) at iterator.jl:409
iteratorsize{T<:Base.IteratorND{I,N}}(::Type{T}) at iterator.jl:453
iteratorsize{I}(::Type{Base.Flatten{I}}) at iterator.jl:481
iteratorsize{I}(::Type{CartesianRange{I}}) at multidimensional.jl:94
iteratorsize{C<:Channel{T}}(::Type{C}) at channels.jl:99
iteratorsize{S,F}(::Type{WordStreams.WordStream{S,F}}) at /mnt/phd/prototypes/SenseSplittingWord2Vec/src/WordStreams.jl:81
iteratorsize(::Type) at generator.jl:37
iteratorsize{T}(::SkipWrapIterator{T}) at In[2]:3
iteratorsize(x) at generator.jl:36

In [ ]:
IOBuffer

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl


Base.AbstractIOBuffer{Array{UInt8,1}}

  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /home/ubuntu/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /home/ubuntu/.julia/v0.5/IJulia/src/execute_request.jl
